In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("cuemath_cleaned_data.csv")

In [30]:
#measures how recently a student logged in?
df['Last_Login'] = pd.to_datetime(df['Last_Login'], dayfirst=True, errors='coerce')

df['days_since_login'] = (pd.to_datetime('today') - df['Last_Login']).dt.days



In [7]:
#create a flag if a student is performing well (based on quiz + final score)?
df['high_performer'] = ((df['quiz_score'] + df['final_grade']) / 2 > 75).astype(int)
df['high_performer'] 

0         0
1         1
2         0
3         1
4         0
         ..
109995    0
109996    0
109997    1
109998    0
109999    0
Name: high_performer, Length: 110000, dtype: int32

In [15]:
#showing progress per time spent?
df['efficiency_score'] = df['weekly_progress(%)'] / df['time_spent_minutes']
df['efficiency_score']

0         1.193766
1         0.100000
2         0.100000
3         0.260365
4         3.105000
            ...   
109995    0.942247
109996    1.024247
109997    0.008611
109998    0.100000
109999    0.100000
Name: efficiency_score, Length: 110000, dtype: float64

In [17]:
#measure student consistency? (weekly progress + login recency)
df['consistency_score'] = df['weekly_progress(%)'] / (df['days_since_login'] + 1)
df['consistency_score']

0         -2.965161
1          0.684615
2          1.284839
3         -0.858696
4          0.241948
            ...    
109995    41.930000
109996     0.983816
109997     0.006019
109998    -0.095833
109999     1.145181
Name: consistency_score, Length: 110000, dtype: float64

In [19]:
#calculate engagement level using multiple features?
df['engagement_score'] = (
    df['time_spent_minutes'] * 0.3 +
    df['weekly_progress(%)'] * 0.4 +
    df['quiz_score'] * 0.3
)
df['engagement_score']

0          78.768
1          57.560
2         147.122
3          90.413
4          30.552
           ...   
109995     85.144
109996     59.308
109997     47.948
109998      8.120
109999    345.370
Name: engagement_score, Length: 110000, dtype: float64

In [20]:
#calculate quiz-to-final-grade improvement rate?
df['improvement_rate'] = df['final_grade'] - df['quiz_score']


In [21]:
#identify students with low feedback but high performance?
df['underrated_flag'] = ((df['final_grade'] > 80) & (df['feedback_score'] < 3)).astype(int)


In [28]:
#measure inactivity duration?
df['Enrollment_Date'] = pd.to_datetime(df['Enrollment_Date'], dayfirst=True, errors='coerce')
df['Last_Login'] = pd.to_datetime(df['Last_Login'], dayfirst=True, errors='coerce')

df['inactive_days'] = (df['Last_Login'] - df['Enrollment_Date']).dt.days


In [26]:
#dropout risk indicator (based on last login + progress)?
df['dropout_risk'] = ((df['days_since_login'] > 30) & (df['weekly_progress(%)'] < 10)).astype(int)


In [27]:
# Grade-Level Categorization:
def categorize_grade(grade):
    if grade in [1, 2, 3]:
        return "Excellent"
    elif grade in [4, 5, 6]:
        return "Very Good"
    elif grade in [7, 8, 9]:
        return "Good"
    elif grade in [10, 11, 12]:
        return "Needs Improvement"
    else:
        return "Unknown"

df["grade_category"] = df["grade"].apply(categorize_grade)


In [31]:
df.head()

,student_id,name,age,quiz_score,city,Enrollment_Date,Last_Login,course_enrolled,weekly_progress(%),final_grade,...,days_since_login,high_performer,efficiency_score,consistency_score,engagement_score,improvement_rate,underrated_flag,dropout_risk,grade_category,inactive_days
0,10000,Riya Chaudry,11,63,Bangalore,2025-05-01,2025-05-17,Probability,91.92,79.2,...,-32,0,1.193766,-2.965161,78.768,16.2,0,0,Excellent,16
1,10001,Vaibhav Bajaj,15,91,Mumbai,2025-03-19,2025-04-03,Probability,8.90,68.5,...,12,1,0.100000,0.684615,57.560,-22.5,0,0,Very Good,15
2,10002,Anya Khurana,5,39,Hyderabad,2025-02-22,2025-03-16,Algebra Basics,39.83,62.7,...,30,0,0.100000,1.284839,147.122,23.7,0,0,Needs Improvement,22
3,10003,Anvi Divan,11,97,Mumbai,2025-05-07,2025-06-01,Math Puzzles,39.50,58.4,...,-47,1,0.260365,-0.858696,90.413,-38.6,0,0,Very Good,25
4,10004,Prisha Maharaj,11,71,Pune,2025-01-26,2025-01-29,Calculus Intro,18.63,67.8,...,76,0,3.105000,0.241948,30.552,-3.2,0,0,Needs Improvement,3
